<a href="https://colab.research.google.com/github/AaryanBairagi/GeekForGeeks-21Days-21Projects-Challenge/blob/main/Day%2015/Day_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3

In [2]:
##CREATED MY 3 DATASETS AND GENERATED THEIR CSV FILES, GOT THESE LINKS FROM MOCKAROO
! curl "https://api.mockaroo.com/api/62be7000?count=1000&key=d458dec0" > "customers.csv"
! curl "https://api.mockaroo.com/api/ad55cfc0?count=1000&key=d458dec0" > "orders.csv"
! curl "https://api.mockaroo.com/api/492b5160?count=1000&key=d458dec0" > "products.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100k    0  100k    0     0  55159      0 --:--:--  0:00:01 --:--:-- 55135
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79333    0 79333    0     0  19772      0 --:--:--  0:00:04 --:--:-- 19783
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3245k    0 3245k    0     0   859k      0 --:--:--  0:00:03 --:--:--  859k


In [3]:
##SETTING UP ALL THE DATAFRAMES
customers = pd.read_csv('customers.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [4]:
##CREATE A SQLITE DATABASE AND STORE THEE TABLES FROM MOCKAROO
db_name = "ecommerce.db"
conn = sqlite3.connect(db_name)

customers.to_sql("customers" , conn , index=False , if_exists="replace")
orders.to_sql("orders" , conn , index=False , if_exists="replace")
products.to_sql("products" , conn , index=False , if_exists="replace")

1000

In [5]:
##TESING THE CONNECTION
pd.read_sql("SELECT * FROM customers LIMIT 5;",conn)

,customer_id,first_name,last_name,email,phone_number,address,city,country,postal_code,membership_level
0,1,Adena,Philbrook,aphilbrook0@newyorker.com,962-605-9500,14 Anthes Drive,Huangyang,China,None,Silver
1,2,Germaine,McTeer,gmcteer1@wordpress.org,811-395-4150,3 Elmside Park,Pathum Rat,Thailand,45190,Silver
2,3,Ernest,Kira,ekira2@admin.ch,175-181-1055,2596 Kedzie Court,Kerek,Indonesia,None,Platinum
3,4,Theda,Bannister,tbannister3@biglobe.ne.jp,520-895-5583,32745 New Castle Drive,Dougou,China,None,Gold
4,5,Minda,Tossell,mtossell4@shop-pro.jp,297-325-3111,9781 Kinsman Pass,Jingyang,China,None,Basic


In [6]:
##IMPORTED GOOGLE API KEY

In [7]:
!pip install google-genai

In [8]:
from google import genai
from google.colab import userdata

In [9]:
genai_client = genai.Client(api_key=userdata.get('ecommerce_db'))

In [10]:
##DEFINING ALL THE FUNCTIONS
import json
def get_sql_query(genaiClient ,prompt,user_query) :
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on :
  {user_query}
  """
  response = genaiClient.models.generate_content(model='gemini-2.5-flash',contents=contents)
  print(response)

  usage_metadata = response.usage_metadata

  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count : {usage_metadata.thoughts_token_count}")
  print(f"Output Token Count : {usage_metadata.candidates_token_count}")
  print(f"Total Token Count : {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json','').replace('```',''))
  return output

In [11]:
def execute_query(query,db_name='ecommerce.db') :
  conn = None
  try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    print(f"\n Executing query on '{db_name}':\n{query}")
    cursor.execute(query)
    results = cursor.fetchall()

    columns = [description[0] for description in cursor.description]

    results_as_dict = [dict(zip(columns,row)) for row in results]
    results_df = pd.DataFrame(results_as_dict)
    print("Query executed successfully.")
    return results_df

  except sqlite3.Error as e:
    print(f"Database error executing query: {e}")
    return None
  except Exception as e:
    print(f"An unexpected error occurred : {e}")
    return None
  finally :
    if conn:
      conn.close()

In [12]:
def text2sql(genaiClient, prompt,user_query) :
  output = get_sql_query(genaiClient,prompt,user_query)
  if(output['status']=='success'):
    results = execute_query(output['response'])
    return results
  return output

In [13]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's e-commerce database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following three tables:

**`customers` table:**

```sql
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    address VARCHAR(50),
    city VARCHAR(50),
    country VARCHAR(50),
    postal_code VARCHAR(50),
    loyalty_points INT
);
```

**`products` table:**

```sql
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name TEXT,
    description TEXT,
    price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2),
    category VARCHAR(50),
    brand TEXT,
    stock_quantity INT,
    color VARCHAR(50),
    size VARCHAR(20),
    weight DECIMAL(5,2),
    dimensions TEXT,
    release_date DATE,
    rating DECIMAL(3,1),
    reviews_count INT,
    seller_name TEXT,
    seller_rating DECIMAL(3,1),
    seller_reviews_count INT,
    shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
```

**`orders` table:**

```sql
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    total_price DECIMAL(10,2),
    order_date DATE,
    shipping_address VARCHAR(255),
    payment_method VARCHAR(20),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Show me all customers who live in Noida"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT * FROM customers WHERE city = 'Noida';"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "What are the names of the top 3 products with the highest total revenue?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Show me recent orders"
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you please define what 'recent' means? For example, 'in the last 7 days', 'this month', or 'since August 2025'."
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Which warehouse has the most stock?"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about warehouses."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

In [14]:
user_query = "What are my most popular products by total quantity sold?"
text2sql(genai_client, prompt, user_query)

sdk_http_response=HttpResponse(
  headers=<dict len=10>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""```json
{
  "status": "success",
  "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.quantity) DESC;"
}
```"""
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='MFbaaJzGDLyez7IP65riqAo' usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=73,
  prompt_token_count=1539,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=1539
    ),
  ],
  thoughts_token_count=88,
  total_token_count=1700
) automatic_function_calling_history=[] parsed=None
Input Token Count: 1539
Thoughts Token Count : 88
Output Token Count : 73
Total Token Count : 170

,product_name
0,LED Strip Lights
1,Dark Chocolate Bars
2,Smartphone Camera Lens Kit
3,Vegetable Lentil Soup
4,Classic Black Trousers
...,...
557,Herb Drying Rack
558,Carrot and Celery Sticks
559,Lentil Soup
560,Cacao Nibs


# **SUBMISSION TASK**

In [15]:
! curl "https://api.mockaroo.com/api/8f5dda90?count=1000&key=d458dec0" > "employees.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100k    0  100k    0     0  49117      0 --:--:--  0:00:02 --:--:-- 49140


In [16]:
employees = pd.read_csv("employees.csv")

In [29]:
db_new_name = "employees.db"

conn_new = sqlite3.connect(db_new_name)

employees.to_sql("employees" , conn_new , index=False , if_exists="replace")


1000

In [30]:
from google import genai
from google.colab import userdata

In [31]:
pd.read_sql("SELECT * FROM employees LIMIT 5;", conn_new)

,employee_id,first_name,last_name,email,phone_number,job_title,department,hire_date,salary,manager_id
0,1,Del,Bagshawe,dbagshawe0@sciencedaily.com,915-226-1345,Senior Sales Associate,Finance,8/23/2004,40840.32,1
1,2,Wyatan,Burridge,wburridge1@sohu.com,887-959-1317,Safety Technician I,Marketing,4/26/2012,129071.02,2
2,3,Rebeca,Hindrich,rhindrich2@pcworld.com,621-155-8386,Automation Specialist III,Marketing,1/31/2009,60318.31,3
3,4,Sheridan,Tatershall,statershall3@bigcartel.com,559-902-6869,Quality Control Specialist,HR,6/11/2009,121202.55,4
4,5,Greer,Kornacki,gkornacki4@51.la,393-559-5733,Statistician I,Finance,7/7/2011,84693.15,5


In [32]:
genai_client_new = genai.Client(api_key=userdata.get('employee_db'))

In [33]:
prompt_employees = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for an HR dashboard. This tool allows non-technical staff and managers to query the company's employees database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following table:

**`employees` table:**

```sql
CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    phone_number VARCHAR(20),
    hire_date DATE,
    job_id VARCHAR(20),
    salary DECIMAL(10,2),
    manager_id INT
);

---

## **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query.

---

## **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

"status": Must be "success", "clarification_needed", or "error".

"response":

If "success", this is a string containing the SQLite query.

If "clarification_needed", this is a string with a clarifying question.

If "error", this is a string explaining why the query could not be generated.

Only output valid JSON. Do not include SQL code fences or extra text.


---

### Example Input → Output

**User Query:**
`"whats the maximum and minimum salary of employees in the db?"`

**Expected Model Output:**
```json
{
  "status": "success",
  "response": "SELECT MAX(salary) AS max_salary, MIN(salary) AS min_salary FROM employees;"
}

"""

In [34]:
user_query_emp = "whats the maximum and minimum salary of employees in the db?"

In [37]:
##DEFINING ALL THE FUNCTIONS
import json
def get_sql_query_new(genaiClient ,prompt,user_query) :
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on :
  {user_query}
  """
  response = genaiClient.models.generate_content(model='gemini-2.5-flash',contents=contents)
  print(response)

  usage_metadata = response.usage_metadata

  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count : {usage_metadata.thoughts_token_count}")
  print(f"Output Token Count : {usage_metadata.candidates_token_count}")
  print(f"Total Token Count : {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json','').replace('```',''))
  return output

In [38]:
def execute_query_new(query,db_name='employees.db') :
  conn = None
  try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    print(f"\n Executing query on '{db_name}':\n{query}")
    cursor.execute(query)
    results = cursor.fetchall()

    columns = [description[0] for description in cursor.description]

    results_as_dict = [dict(zip(columns,row)) for row in results]
    results_df = pd.DataFrame(results_as_dict)
    print("Query executed successfully.")
    return results_df

  except sqlite3.Error as e:
    print(f"Database error executing query: {e}")
    return None
  except Exception as e:
    print(f"An unexpected error occurred : {e}")
    return None
  finally :
    if conn:
      conn.close()

In [39]:
def text2sql_new(genaiClient, prompt,user_query) :
  output = get_sql_query_new(genaiClient,prompt,user_query)
  if(output['status']=='success'):
    results = execute_query_new(output['response'])
    return results
  return output

In [40]:
text2sql_new(genai_client_new , prompt_employees , user_query_emp)

sdk_http_response=HttpResponse(
  headers=<dict len=10>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""```json
{
  "status": "success",
  "response": "SELECT MAX(salary) AS max_salary, MIN(salary) AS min_salary FROM employees;"
}
```"""
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='vVfaaJK7NIbQz7IP85jWmQs' usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=43,
  prompt_token_count=478,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=478
    ),
  ],
  thoughts_token_count=95,
  total_token_count=616
) automatic_function_calling_history=[] parsed=None
Input Token Count: 478
Thoughts Token Count : 95
Output Token Count : 43
Total Token Count : 616

 Executing query on 'employees.db':
SELECT MAX(salary) AS max_salary, MIN(salary) A

,max_salary,min_salary
0,149772.48,30013.04
